1. Tiền xử lý dữ liệu

In [20]:
#Đọc dữ liệu song ngữ
with open(r'C:\Users\phamq\Downloads\BTL - Xử lý ngôn ngữ tự nhiên\data\train.en', 'r', encoding='utf-8') as f:
    en_sentences = f.read().strip().split('\n')

with open(r'C:\Users\phamq\Downloads\BTL - Xử lý ngôn ngữ tự nhiên\data\train.vi', 'r', encoding='utf-8') as f:
    vi_sentences = f.read().strip().split('\n')

In [21]:
#làm sạch
import re

def clean_sentence(s):
    s = s.lower().strip()               #lower chuyển tất cả các chữ thành chữ thường, strip xóa khoảng trắng đầu và cuối câu
    s = re.sub(r"([?.!,])", r" \1 ", s) #thêm dấu cách trước và sau các dấu câu
    s = re.sub(r'[" "]+', " ", s)       #loại bỏ các khoảng trắng thừa
    s = re.sub(r"[^a-zA-Z?.!,’'`àáảãạăằắẳẵặâầấẩẫậèéẻẽẹêềếểễệìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵđ]", " ", s) #loại bỏ các ký tự không cần thiết
    s = s.strip() 
    return s

#làm sạch toàn bộ các câu song ngữ trước khi đưa vào mô hình
en_sentences = [clean_sentence(s) for s in en_sentences]
vi_sentences = [clean_sentence(s) for s in vi_sentences]


In [22]:
vi_sentences = ['<start> ' + s + ' <end>' for s in vi_sentences] #thêm token start và end cho các câu tiếng Việt

In [23]:
#Tạo token cho các câu, chuyển các câu thành token ID
from tensorflow.keras.preprocessing.text import Tokenizer

#Tiếng Anh
en_tokenizer = Tokenizer(filters='')
en_tokenizer.fit_on_texts(en_sentences)
en_sequences = en_tokenizer.texts_to_sequences(en_sentences)

#Tiếng Việt
vi_tokenizer = Tokenizer(filters='')
vi_tokenizer.fit_on_texts(vi_sentences)
vi_sequences = vi_tokenizer.texts_to_sequences(vi_sentences)

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

#độ dài trong danh sách tiếng anh và tiếng việt
MAX_LEN = 80

en_padded = pad_sequences(en_sequences, maxlen=MAX_LEN, padding='post', truncating='post')
vi_padded = pad_sequences(vi_sequences, maxlen=MAX_LEN, padding='post', truncating='post')


#áp dụng padding cho tất cả chuỗi senquences
#Chỉ dùng 50.000 câu để thử
en_sequences_small = en_sequences[:50000]
vi_sequences_small = vi_sequences[:50000]

en_padded = pad_sequences(en_sequences_small, maxlen=MAX_LEN, padding='post', truncating='post')
vi_padded = pad_sequences(vi_sequences_small, maxlen=MAX_LEN, padding='post', truncating='post')


In [25]:
#Chia ra hai phần train và valid (đánh giá) 
from sklearn.model_selection import train_test_split

en_train, en_val, vi_train, vi_val = train_test_split(en_padded, vi_padded, test_size=0.1)

In [26]:
#Tạo data set
import tensorflow as tf

BUFFER_SIZE = len(en_train)
BATCH_SIZE = 64

train_dataset = tf.data.Dataset.from_tensor_slices((en_train, vi_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

val_dataset = tf.data.Dataset.from_tensor_slices((en_val, vi_val))
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

2. Xây dựng mô hình

In [27]:
#Định nghĩa mô hình Encoder, Attention, Decoder
from tensorflow.keras.layers import Embedding, GRU, Dense, Input, Layer

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.gru = GRU(enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self, batch_sz):
        return tf.zeros((batch_sz, self.enc_units))

class BahdanauAttention(Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, query, values):
        # query shape: (batch_size, hidden size)
        # values shape: (batch_size, max_len, hidden size)
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(query_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.gru = GRU(dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
        self.fc = Dense(vocab_size)

        self.attention = BahdanauAttention(dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)

        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        output, state = self.gru(x)

        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)

        return x, state, attention_weights


In [28]:
#Khởi tạo mô hình
embedding_dim = 64
units = 128
vocab_inp_size = len(en_tokenizer.word_index) + 1   
vocab_tar_size = len(vi_tokenizer.word_index) + 1

encoder = Encoder(vocab_inp_size, embedding_dim, units)
decoder = Decoder(vocab_tar_size, embedding_dim, units)

3. Huấn luyện mô hình

In [29]:
#Định nghĩa optimizer và loss function
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [30]:
BUFFER_SIZE = len(en_padded)
BATCH_SIZE = 64
steps_per_epoch = BUFFER_SIZE // BATCH_SIZE

dataset = tf.data.Dataset.from_tensor_slices((en_padded, vi_padded))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [31]:
start_token = tf.constant(vi_tokenizer.word_index['<start>'], dtype=tf.int32)

@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden

        # Tạo dec_input bằng Tensor constant
        dec_input = tf.expand_dims(tf.repeat(start_token, BATCH_SIZE), 1)

        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = loss / tf.cast(targ.shape[1], tf.float32)
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss


In [ ]:
EPOCHS = 40

from tqdm import tqdm

for epoch in range(EPOCHS):                                                 #Bắt đầu vòng lặp
    enc_hidden = encoder.initialize_hidden_state(BATCH_SIZE)
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

    print(f'Epoch {epoch+1} Loss {total_loss / steps_per_epoch:.4f}')
    
# Lưu mô hình mỗi epoch
encoder.save_weights(f'encoder_epoch{epoch+1}.weights.h5')
decoder.save_weights(f'decoder_epoch{epoch+1}.weights.h5')

Epoch 1 Loss 1.2210
Epoch 2 Loss 1.1623
Epoch 3 Loss 1.1251
Epoch 4 Loss 1.1146
Epoch 5 Loss 1.0481
Epoch 6 Loss 1.0573
Epoch 7 Loss 1.0111
Epoch 8 Loss 1.0157
Epoch 9 Loss 0.9920
Epoch 10 Loss 0.9568


4. Đánh giá huấn luyện

In [74]:
import numpy as np
import tensorflow as tf
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

encoder.load_weights(r'C:\Users\phamq\Downloads\BTL - Xử lý ngôn ngữ tự nhiên\encoder_epoch10.weights.h5')
decoder.load_weights(r'C:\Users\phamq\Downloads\BTL - Xử lý ngôn ngữ tự nhiên\decoder_epoch10.weights.h5')

# Lấy tập validation từ 10% cuối
val_input_sentences = en_sentences[-100:]
val_target_sentences = vi_sentences[-100:]

# Tiền xử lý 1 câu
def preprocess_sentence(sentence):
    import numpy as np
    if isinstance(sentence, np.ndarray):
        if sentence.ndim == 0:
            sentence = sentence.item()
        else:
            sentence = sentence[0]
    elif isinstance(sentence, list):
        if isinstance(sentence[0], str):
            sentence = sentence[0]
        else:
            sentence = ' '.join(str(w) for w in sentence)
    return sentence.lower().strip()

# Hàm dịch một câu
def evaluate(sentence):
    sentence = preprocess_sentence(sentence)
    inputs = en_tokenizer.texts_to_sequences([sentence])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''
    hidden = encoder.initialize_hidden_state(1)
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([vi_tokenizer.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        predicted_word = vi_tokenizer.index_word.get(predicted_id, '')

        if predicted_word == '<end>':
            break
        result += predicted_word + ' '
        dec_input = tf.expand_dims([predicted_id], 0)

    return result.strip()

# Tính BLEU cho 1 cặp câu
def compute_bleu(reference, hypothesis):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference], hypothesis, smoothing_function=smoothie)

# Đánh giá trên tập validation
bleu_scores = []

print("Đang đánh giá mô hình trên tập validation...\n")
for i in range(len(val_input_sentences)):
    input_sentence = val_input_sentences[i]
    target_sentence = val_target_sentences[i]

    pred_sentence = evaluate(input_sentence)

    ref = preprocess_sentence(target_sentence).split()
    hyp = pred_sentence.split()

    score = compute_bleu(ref, hyp)
    bleu_scores.append(score)

    # In 10 câu đầu để kiểm tra
    if i < 10:
        print(f"[{i+1}] Input (EN):     {input_sentence}")
        print(f"    Target (VI):    {target_sentence}")
        print(f"    Predicted (VI): {pred_sentence}")
        print(f"    BLEU score:     {score:.4f}")
        print("-" * 60)

# BLEU trung bình
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"\n🎯 Điểm BLEU trung bình trên tập validation: {average_bleu:.4f}")

Đang đánh giá mô hình trên tập validation...

[1] Input (EN):     they apos re actually about improving houses .
    Target (VI):    start mà là cải thiện nhà cửa . end
    Predicted (VI): họ thực hiện ra những nhà thiết kế .
    BLEU score:     0.0306
------------------------------------------------------------
[2] Input (EN):     we start on day one of every project we apos ve learned , we don apos t make promises , we don apos t do reports .
    Target (VI):    start mỗi ngày chúng tôi làm dự án theo kinh nghiệm , chúng tôi không hứa , không báo cáo . end
    Predicted (VI): chúng tôi bắt đầu đi ngày , một nhóm người ta đã học được , chúng ta không thể làm những ý tưởng , chúng ta không làm việc đào tạo .
    BLEU score:     0.0343
------------------------------------------------------------
[3] Input (EN):     we arrive in the morning with tools , tons of equipment , trades , and we train up a local team on the first day to start work .
    Target (VI):    start mỗi sáng chúng tôi 

5. Demo web app

In [77]:
import nest_asyncio
nest_asyncio.apply()

from flask import Flask, request, render_template_string

app = Flask(__name__)

# Hàm dịch một câu
def evaluate(sentence):
    sentence = preprocess_sentence(sentence)
    inputs = en_tokenizer.texts_to_sequences([sentence])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''
    hidden = encoder.initialize_hidden_state(1)
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([vi_tokenizer.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        predicted_word = vi_tokenizer.index_word.get(predicted_id, '')

        if predicted_word == '<end>':
            break
        result += predicted_word + ' '
        dec_input = tf.expand_dims([predicted_id], 0)

    return result.strip()

html = """
<!DOCTYPE html>
<html lang="en">
<head><meta charset="UTF-8"><title>Demo Dịch Máy</title></head>
<body>
<h1>Demo Dịch Máy Anh-Việt</h1>
<form method="POST">
    <textarea name="input_text" rows="3" cols="50" required></textarea><br/>
    <button type="submit">Dịch</button>
</form>
{% if translation %}
<h2>Kết quả dịch:</h2>
<p>{{ translation }}</p>
{% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    translation = ""
    if request.method == "POST":
        input_text = request.form["input_text"]
        translation = evaluate(input_text)
    return render_template_string(html, translation=translation)

if __name__ == "__main__":
    app.run(port=5000, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Jun/2025 07:12:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2025 07:12:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Jun/2025 07:12:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2025 07:12:56] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2025 07:13:58] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2025 07:14:05] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2025 07:14:15] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2025 07:14:18] "POST / HTTP/1.1" 200 -
